# About this classifier
This is example for UrbanSound8K classification based on "[End-to-end baseline TF Estimator LB0.72](https://www.kaggle.com/alexozerin/end-to-end-baseline-tf-estimator-lb-0-72)" at Keggle competition.

Before applying to UrbanSound8K, prepared followings.
- Cleaning all the wave file in UrbanSound8K to be 16kHz & 16bit format to make it consistent. It was verrrry slow to clense audio format online. Cleaning everything in advance made it 10 times or more faster.
- Preparing data lists for training set and validation set.

The original example uses [TF Estimator](https://www.tensorflow.org/extend/estimators) high-level API framework, the changes made here are:
- Loading datasets is for UrbanSound8K.
- data_generator to fit to UrbanSound problem.

So how's the accuracy? It's about 55%, this is done even without fitting model and learning etc to the UrbanSound problem.

Other notebooks:
- To create data lists: [1_urban_sound_validation_lister.ipynb](1_urban_sound_validation_lister.ipynb)
- To clean all wave files: [2_urban_sound_convert_to_16kmono.ipynb](2_urban_sound_convert_to_16kmono.ipynb)

In [1]:
import os
import re
import pandas as pd
DATADIR = './16mUrbanSound8K'
OUTDIR = './result' # just a random name

POSSIBLE_LABELS = [str(i) for i in range(10)]
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}
sample_rate = 16000

def load_data():
    """ Return 2 lists of tuples:
    [(class_id, path), ...] for train
    [(class_id, path), ...] for validation
    """
    #alldata = pd.read_csv(os.path.join(DATADIR, 'metadata', 'UrbanSound8K.csv'))
    traindata = pd.read_csv(os.path.join('.', 'train_list.csv'))
    valdata = pd.read_csv(os.path.join('.', 'validation_list.csv'))
    trainset = [(int(traindata.classID[k]), os.path.join(DATADIR, 'audio/fold%d/%s' % (traindata.fold[k], traindata.slice_file_name[k])))
                for k in range(len(traindata))]
    valset = [(int(valdata.classID[k]), os.path.join(DATADIR, 'audio/fold%d/%s' % (valdata.fold[k], valdata.slice_file_name[k])))
              for k in range(len(valdata))]
    return trainset, valset

trainset, valset = load_data()
print('trainset = %d samples.' % len(trainset))
print('valset = %d samples.' % len(valset))

trainset = 6988 samples.
valset = 1744 samples.


In [2]:
import numpy as np
from scipy.io import wavfile

def data_generator(data, params, mode='train'):
    def generator():
        if mode == 'train':
            np.random.shuffle(data)
        # Feel free to add any augmentation
        for (label_id, fname) in data:
            try:
                _, wav = wavfile.read(fname)
                wav = wav.astype(np.float32) / np.iinfo(np.int16).max

                L = 16000  # be aware, some files are shorter than 1 sec!
                if len(wav) < L:
                    continue
                if len(wav) > L:
                    beg = np.random.randint(0, len(wav) - L)
                else:
                    beg = 0
                yield dict(
                    target=np.int32(label_id),
                    wav=wav[beg: beg + L],
                )
            except Exception as err:
                print('data_generator error', err, label_id, fname)

    return generator

In [3]:
import tensorflow as tf
from tensorflow.contrib import layers

def baseline(x, params, is_training):
    x = layers.batch_norm(x, is_training=is_training)
    for i in range(4):
        x = layers.conv2d(
            x, 16 * (2 ** i), 3, 1,
            activation_fn=tf.nn.elu,
            normalizer_fn=layers.batch_norm if params.use_batch_norm else None,
            normalizer_params={'is_training': is_training}
        )
        x = layers.max_pool2d(x, 2, 2)

    # just take two kind of pooling and then mix them, why not :)
    mpool = tf.reduce_max(x, axis=[1, 2], keep_dims=True)
    apool = tf.reduce_mean(x, axis=[1, 2], keep_dims=True)

    x = 0.5 * (mpool + apool)
    # we can use conv2d 1x1 instead of dense
    x = layers.conv2d(x, 128, 1, 1, activation_fn=tf.nn.elu)
    x = tf.nn.dropout(x, keep_prob=params.keep_prob if is_training else 1.0)
    
    # again conv2d 1x1 instead of dense layer
    logits = layers.conv2d(x, params.num_classes, 1, 1, activation_fn=None)
    return tf.squeeze(logits, [1, 2])

/home/niz/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
from tensorflow.contrib import signal

# features is a dict with keys: tensors from our datagenerator
# labels also were in features, but excluded in generator_input_fn by target_key

def model_handler(features, labels, mode, params, config):
    # Im really like to use make_template instead of variable_scopes and re-usage
    extractor = tf.make_template(
        'extractor', baseline,
        create_scope_now_=True,
    )
    # wav is a waveform signal with shape (16000, )
    wav = features['wav']
    # we want to compute spectograms by means of short time fourier transform:
    specgram = signal.stft(
        wav,
        400,  # 16000 [samples per second] * 0.025 [s] -- default stft window frame
        160,  # 16000 * 0.010 -- default stride
    )
    # specgram is a complex tensor, so split it into abs and phase parts:
    phase = tf.angle(specgram) / np.pi
    # log(1 + abs) is a default transformation for energy units
    amp = tf.log1p(tf.abs(specgram))
    
    x = tf.stack([amp, phase], axis=3) # shape is [bs, time, freq_bins, 2]
    x = tf.to_float(x)  # we want to have float32, not float64

    logits = extractor(x, params, mode == tf.estimator.ModeKeys.TRAIN)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        # some lr tuner, you could use move interesting functions
        def learning_rate_decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(
                learning_rate, global_step, decay_steps=10000, decay_rate=0.99)

        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer=lambda lr: tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True),
            learning_rate_decay_fn=learning_rate_decay_fn,
            clip_gradients=params.clip_gradients,
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )

    if mode == tf.estimator.ModeKeys.EVAL:
        prediction = tf.argmax(logits, axis=-1)
        acc, acc_op = tf.metrics.mean_per_class_accuracy(
            labels, prediction, params.num_classes)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops=dict(
                acc=(acc, acc_op),
            )
        )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'label': tf.argmax(logits, axis=-1),  # for probability just take tf.nn.softmax()
            'sample': features['sample'], # it's a hack for simplicity
        }
        specs = dict(
            mode=mode,
            predictions=predictions,
        )
    return tf.estimator.EstimatorSpec(**specs)


def create_model(config=None, hparams=None):
    return tf.estimator.Estimator(
        model_fn=model_handler,
        config=config,
        params=hparams,
    )

In [5]:
params=dict(
    seed=2018,
    batch_size=64,
    keep_prob=0.5,
    learning_rate=1e-3,
    clip_gradients=15.0,
    use_batch_norm=True,
    num_classes=len(POSSIBLE_LABELS),
)

hparams = tf.contrib.training.HParams(**params)
os.makedirs(os.path.join(OUTDIR, 'eval'), exist_ok=True)
model_dir = OUTDIR

run_config = tf.contrib.learn.RunConfig(model_dir=model_dir)

In [6]:
# it's a magic function :)
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn
            
train_input_fn = generator_input_fn(
    x=data_generator(trainset, hparams, 'train'),
    target_key='target',  # you could leave target_key in features, so labels in model_handler will be empty
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)

val_input_fn = generator_input_fn(
    x=data_generator(valset, hparams, 'val'),
    target_key='target',
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)
            

def _create_my_experiment(run_config, hparams):
    exp = tf.contrib.learn.Experiment(
        estimator=create_model(config=run_config, hparams=hparams),
        train_input_fn=train_input_fn,
        eval_input_fn=val_input_fn,
        train_steps=20000, # just randomly selected params
        eval_steps=200,  # read source code for steps-epochs ariphmetics
        train_steps_per_iteration=1000,
    )
    return exp

tf.contrib.learn.learn_runner.run(
    experiment_fn=_create_my_experiment,
    run_config=run_config,
    schedule="continuous_train_and_eval",
    hparams=hparams)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4b414e8b70>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './result'}
INFO:tensorflow:Training model for 1000 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./result/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into ./result/model.ckpt.
INFO:tensorflow:loss = 4.67352, step = 2
INFO:tensorflow:global_step/sec: 10.5691
INFO:tensorflow:loss = 1.77638, step = 102 (9.465 sec)
INFO:tens

INFO:tensorflow:Evaluation [155/200]
INFO:tensorflow:Evaluation [156/200]
INFO:tensorflow:Evaluation [157/200]
INFO:tensorflow:Evaluation [158/200]
INFO:tensorflow:Evaluation [159/200]
INFO:tensorflow:Evaluation [160/200]
INFO:tensorflow:Evaluation [161/200]
INFO:tensorflow:Evaluation [162/200]
INFO:tensorflow:Evaluation [163/200]
INFO:tensorflow:Evaluation [164/200]
INFO:tensorflow:Evaluation [165/200]
INFO:tensorflow:Evaluation [166/200]
INFO:tensorflow:Evaluation [167/200]
INFO:tensorflow:Evaluation [168/200]
INFO:tensorflow:Evaluation [169/200]
INFO:tensorflow:Evaluation [170/200]
INFO:tensorflow:Evaluation [171/200]
INFO:tensorflow:Evaluation [172/200]
INFO:tensorflow:Evaluation [173/200]
INFO:tensorflow:Evaluation [174/200]
INFO:tensorflow:Evaluation [175/200]
INFO:tensorflow:Evaluation [176/200]
INFO:tensorflow:Evaluation [177/200]
INFO:tensorflow:Evaluation [178/200]
INFO:tensorflow:Evaluation [179/200]
INFO:tensorflow:Evaluation [180/200]
INFO:tensorflow:Evaluation [181/200]
I

INFO:tensorflow:Evaluation [137/200]
INFO:tensorflow:Evaluation [138/200]
INFO:tensorflow:Evaluation [139/200]
INFO:tensorflow:Evaluation [140/200]
INFO:tensorflow:Evaluation [141/200]
INFO:tensorflow:Evaluation [142/200]
INFO:tensorflow:Evaluation [143/200]
INFO:tensorflow:Evaluation [144/200]
INFO:tensorflow:Evaluation [145/200]
INFO:tensorflow:Evaluation [146/200]
INFO:tensorflow:Evaluation [147/200]
INFO:tensorflow:Evaluation [148/200]
INFO:tensorflow:Evaluation [149/200]
INFO:tensorflow:Evaluation [150/200]
INFO:tensorflow:Evaluation [151/200]
INFO:tensorflow:Evaluation [152/200]
INFO:tensorflow:Evaluation [153/200]
INFO:tensorflow:Evaluation [154/200]
INFO:tensorflow:Evaluation [155/200]
INFO:tensorflow:Evaluation [156/200]
INFO:tensorflow:Evaluation [157/200]
INFO:tensorflow:Evaluation [158/200]
INFO:tensorflow:Evaluation [159/200]
INFO:tensorflow:Evaluation [160/200]
INFO:tensorflow:Evaluation [161/200]
INFO:tensorflow:Evaluation [162/200]
INFO:tensorflow:Evaluation [163/200]
I

INFO:tensorflow:Evaluation [119/200]
INFO:tensorflow:Evaluation [120/200]
INFO:tensorflow:Evaluation [121/200]
INFO:tensorflow:Evaluation [122/200]
INFO:tensorflow:Evaluation [123/200]
INFO:tensorflow:Evaluation [124/200]
INFO:tensorflow:Evaluation [125/200]
INFO:tensorflow:Evaluation [126/200]
INFO:tensorflow:Evaluation [127/200]
INFO:tensorflow:Evaluation [128/200]
INFO:tensorflow:Evaluation [129/200]
INFO:tensorflow:Evaluation [130/200]
INFO:tensorflow:Evaluation [131/200]
INFO:tensorflow:Evaluation [132/200]
INFO:tensorflow:Evaluation [133/200]
INFO:tensorflow:Evaluation [134/200]
INFO:tensorflow:Evaluation [135/200]
INFO:tensorflow:Evaluation [136/200]
INFO:tensorflow:Evaluation [137/200]
INFO:tensorflow:Evaluation [138/200]
INFO:tensorflow:Evaluation [139/200]
INFO:tensorflow:Evaluation [140/200]
INFO:tensorflow:Evaluation [141/200]
INFO:tensorflow:Evaluation [142/200]
INFO:tensorflow:Evaluation [143/200]
INFO:tensorflow:Evaluation [144/200]
INFO:tensorflow:Evaluation [145/200]
I

INFO:tensorflow:Evaluation [101/200]
INFO:tensorflow:Evaluation [102/200]
INFO:tensorflow:Evaluation [103/200]
INFO:tensorflow:Evaluation [104/200]
INFO:tensorflow:Evaluation [105/200]
INFO:tensorflow:Evaluation [106/200]
INFO:tensorflow:Evaluation [107/200]
INFO:tensorflow:Evaluation [108/200]
INFO:tensorflow:Evaluation [109/200]
INFO:tensorflow:Evaluation [110/200]
INFO:tensorflow:Evaluation [111/200]
INFO:tensorflow:Evaluation [112/200]
INFO:tensorflow:Evaluation [113/200]
INFO:tensorflow:Evaluation [114/200]
INFO:tensorflow:Evaluation [115/200]
INFO:tensorflow:Evaluation [116/200]
INFO:tensorflow:Evaluation [117/200]
INFO:tensorflow:Evaluation [118/200]
INFO:tensorflow:Evaluation [119/200]
INFO:tensorflow:Evaluation [120/200]
INFO:tensorflow:Evaluation [121/200]
INFO:tensorflow:Evaluation [122/200]
INFO:tensorflow:Evaluation [123/200]
INFO:tensorflow:Evaluation [124/200]
INFO:tensorflow:Evaluation [125/200]
INFO:tensorflow:Evaluation [126/200]
INFO:tensorflow:Evaluation [127/200]
I

...

INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [81/200]
INFO:tensorflow:Evaluation [82/200]
INFO:tensorflow:Evaluation [83/200]
INFO:tensorflow:Evaluation [84/200]
INFO:tensorflow:Evaluation [85/200]
INFO:tensorflow:Evaluation [86/200]
INFO:tensorflow:Evaluation [87/200]
INFO:tensorflow:Evaluation [88/200]
INFO:tensorflow:Evaluation [89/200]
INFO:tensorflow:Evaluation [90/200]
INFO:tensorflow:Evaluation [91/200]
INFO:tensorflow:Evaluation [92/200]
INFO:tensorflow:Evaluation [93/200]
INFO:tensorflow:Evaluation [94/200]
INFO:tensorflow:Evaluation [95/200]
INFO:tensorflow:Evaluation [96/200]
INFO:tensorflow:Evaluation [97/200]
INFO:tensorflow:Evaluation [98/200]
INFO:tensorflow:Evaluation [99/200]
INFO:tensorflow:Evaluation [100/200]
INFO:tensorflow:Evaluation [101/200]
INFO:tensorflow:Evaluation [102/200]
INFO:tensorflow:Evaluation [103/200]
INFO:tensorflow:Evaluation [104/200]
INFO:tensorflow:Evaluation [105/200]
INFO:tensorflow:Evaluation [106/200]
INFO:tensorflow:Evalu

INFO:tensorflow:Evaluation [61/200]
INFO:tensorflow:Evaluation [62/200]
INFO:tensorflow:Evaluation [63/200]
INFO:tensorflow:Evaluation [64/200]
INFO:tensorflow:Evaluation [65/200]
INFO:tensorflow:Evaluation [66/200]
INFO:tensorflow:Evaluation [67/200]
INFO:tensorflow:Evaluation [68/200]
INFO:tensorflow:Evaluation [69/200]
INFO:tensorflow:Evaluation [70/200]
INFO:tensorflow:Evaluation [71/200]
INFO:tensorflow:Evaluation [72/200]
INFO:tensorflow:Evaluation [73/200]
INFO:tensorflow:Evaluation [74/200]
INFO:tensorflow:Evaluation [75/200]
INFO:tensorflow:Evaluation [76/200]
INFO:tensorflow:Evaluation [77/200]
INFO:tensorflow:Evaluation [78/200]
INFO:tensorflow:Evaluation [79/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [81/200]
INFO:tensorflow:Evaluation [82/200]
INFO:tensorflow:Evaluation [83/200]
INFO:tensorflow:Evaluation [84/200]
INFO:tensorflow:Evaluation [85/200]
INFO:tensorflow:Evaluation [86/200]
INFO:tensorflow:Evaluation [87/200]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [42/200]
INFO:tensorflow:Evaluation [43/200]
INFO:tensorflow:Evaluation [44/200]
INFO:tensorflow:Evaluation [45/200]
INFO:tensorflow:Evaluation [46/200]
INFO:tensorflow:Evaluation [47/200]
INFO:tensorflow:Evaluation [48/200]
INFO:tensorflow:Evaluation [49/200]
INFO:tensorflow:Evaluation [50/200]
INFO:tensorflow:Evaluation [51/200]
INFO:tensorflow:Evaluation [52/200]
INFO:tensorflow:Evaluation [53/200]
INFO:tensorflow:Evaluation [54/200]
INFO:tensorflow:Evaluation [55/200]
INFO:tensorflow:Evaluation [56/200]
INFO:tensorflow:Evaluation [57/200]
INFO:tensorflow:Evaluation [58/200]
INFO:tensorflow:Evaluation [59/200]
INFO:tensorflow:Evaluation [60/200]
INFO:tensorflow:Evaluation [61/200]
INFO:tensorflow:Evaluation [62/200]
INFO:tensorflow:Evaluation [63/200]
INFO:tensorflow:Evaluation [64/200]
INFO:tensorflow:Evaluation [65/200]
INFO:tensorflow:Evaluation [66/200]
INFO:tensorflow:Evaluation [67/200]
INFO:tensorflow:Evaluation [68/200]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [23/200]
INFO:tensorflow:Evaluation [24/200]
INFO:tensorflow:Evaluation [25/200]
INFO:tensorflow:Evaluation [26/200]
INFO:tensorflow:Evaluation [27/200]
INFO:tensorflow:Evaluation [28/200]
INFO:tensorflow:Evaluation [29/200]
INFO:tensorflow:Evaluation [30/200]
INFO:tensorflow:Evaluation [31/200]
INFO:tensorflow:Evaluation [32/200]
INFO:tensorflow:Evaluation [33/200]
INFO:tensorflow:Evaluation [34/200]
INFO:tensorflow:Evaluation [35/200]
INFO:tensorflow:Evaluation [36/200]
INFO:tensorflow:Evaluation [37/200]
INFO:tensorflow:Evaluation [38/200]
INFO:tensorflow:Evaluation [39/200]
INFO:tensorflow:Evaluation [40/200]
INFO:tensorflow:Evaluation [41/200]
INFO:tensorflow:Evaluation [42/200]
INFO:tensorflow:Evaluation [43/200]
INFO:tensorflow:Evaluation [44/200]
INFO:tensorflow:Evaluation [45/200]
INFO:tensorflow:Evaluation [46/200]
INFO:tensorflow:Evaluation [47/200]
INFO:tensorflow:Evaluation [48/200]
INFO:tensorflow:Evaluation [49/200]
INFO:tensorflow:Evaluation [

({'acc': 0.57227993, 'global_step': 20001, 'loss': 2.2274594}, [])